In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, date, timedelta 

from matplotlib import pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [2]:
# Licencee name removed with error shown
# to run this code, require Stata 17 and above with Python integration
import stata_setup
stata_setup.config("C:/Program Files/Stata18/", "be")

import pystata
from pystata import stata
from sfi import Scalar, Matrix, Data


  ___  ____  ____  ____  ____ ®
 /__    /   ____/   /   ____/      Stata 18.0
___/   /   /___/   /   /___/       BE—Basic Edition

 Statistics and Data Science       Copyright 1985-2023 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-782-8272        https://www.stata.com
                                   979-696-4600        service@stata.com

Stata license: Single-user  perpetual
Serial number: 301806354400
  Licensed to: Eric Tham
               NTU

Notes:
      1. Unicode is supported; see help unicode_advice.


In [3]:
def setDate(df, start, end):
    start = datetime.strptime(start+"-01","%Y-%m-%d")
    end = datetime.strptime(end+"-01","%Y-%m-%d")
    df["Yr_Mth1"] = df["Yr_Mth"].map(lambda x : datetime.strptime(x+"-01","%Y-%m-%d"))
    df = df.loc[(df["Yr_Mth1"] <= end) & (df["Yr_Mth1"] >= start)]
    return df

In [4]:
def checkDuplicates(df):
    dfa1 = df.loc[:,["permid","Yr_Mth"]]
    dfa2 = dfa1.groupby(["permid","Yr_Mth"])["permid"].count()
    df_duplicates = dfa2.loc[:,dfa2 > 1]  # len 18
    return df_duplicates

In [5]:
pwd = "drive path"
#print(os.getcwd())

### Carbon Future prices and other instrumental variable returns

List of variables:
- MO : Symbol for EUA futures price
- CO : Symbol for Brent future price
- CBOT: Chicago Board of Trade ehtanol price
- CL : WTI price
- TTF : Natural gas price benchmark in Europe

In [6]:
df_carbon_3m = pd.read_csv("final data\\df_carbon_3m.csv")
df_carbon_3m.head()

,Yr_Mth,mo1_price,mo3_price,mo12_price,cl1_price,cl3_price,cl12_price,co1_price,co3_price,co12_price,cbot_eth_q1,cbot_eth_q4,ttf_m1,ttf_m3
0,2005-04,19.119000,19.118000,NaN,53.179184,55.005918,55.107959,52.750208,53.973125,53.536042,NaN,NaN,NaN,NaN
1,2005-07,23.135606,23.125000,NaN,63.312188,64.538281,64.682187,61.908769,63.050462,63.215385,NaN,NaN,NaN,NaN
2,2005-10,21.756923,21.757692,NaN,60.046774,60.934032,61.878387,57.760937,58.954687,60.379375,NaN,NaN,NaN,NaN
3,2006-01,25.554615,25.585385,NaN,63.476774,65.360161,67.542419,62.711094,63.838594,65.942969,NaN,NaN,NaN,NaN
4,2006-04,18.560000,18.973846,20.3,70.722698,72.565556,73.997302,70.361587,71.591905,72.649683,NaN,NaN,NaN,NaN


In [7]:
df_carbon_3m_ret = pd.read_csv("final data\\df_carbon_3m_ret.csv")
df_carbon_3m_ret.loc[df_carbon_3m["mo1_price"] > 0.8, "mo1_price"]  = 0.8  
df_carbon_3m_ret.loc[df_carbon_3m["mo1_price"] < -0.8, "mo1_price"] = -0.8
df_carbon_3m_ret = df_carbon_3m_ret.rename(columns = {"mo1_price": "mo1_ret", "mo3_price": "mo3_ret", \
    "mo12_price":"mo12_ret", "cl1_price":"cl1_ret", "cl3_price": "cl3_ret","cl12_price": "cl12_ret", \
    "co1_price":"co1_ret", "co3_price":"co3_ret","co12_price":"co12_ret", \
    "cbot_eth_q1":"cbot_eth_q1_ret","cbot_eth_q4":"cbot_eth_q4_ret","ttf_m1":"ttf_m1_ret","ttf_m3":"ttf_m3_ret"})
df_carbon_3m_ret.head()

,Yr_Mth,mo1_ret,mo3_ret,mo12_ret,cl1_ret,cl3_ret,cl12_ret,co1_ret,co3_ret,co12_ret,cbot_eth_q1_ret,cbot_eth_q4_ret,ttf_m1_ret,ttf_m3_ret
0,2005-04,0.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2005-07,0.8,0.209593,NaN,0.190545,0.173297,0.173736,0.173621,0.168183,0.180800,NaN,NaN,NaN,NaN
2,2005-10,0.8,-0.059127,NaN,-0.051576,-0.055847,-0.043347,-0.066999,-0.064960,-0.044863,NaN,NaN,NaN,NaN
3,2006-01,0.8,0.175924,NaN,0.057122,0.072638,0.091535,0.085701,0.082842,0.092144,NaN,NaN,NaN,NaN
4,2006-04,0.8,-0.258411,NaN,0.114151,0.110241,0.095568,0.121996,0.121452,0.101705,NaN,NaN,NaN,NaN


## Read in stock data
For both US and Germany
- Returns
- Fundamental data
    - Momentum
    - Book to mkt value ratio
    - Mkt capitalisation
    - Profit indicators: REQ (for EU) retained earnings &  (for US)
- ESG sentiment data: Emissions and EnvironmentalInnovation
- Class: brown, green or consumer

In [8]:
df_gr = pd.read_csv("final data\\final_data_gr.csv")
df_gr = setDate(df_gr, "2006-01", "2022-02")
df_gr.head()

,Unnamed: 0,Yr_Mth,permid,MomentumL1Y,Mth_Ret,mkt_cap,req,bm,Emissions,EnvironmentalInnovation,buzz,class,Yr_Mth1
18,18,2006-01,4295868725,0.066941,0.101909,22.850595,NaN,123.235305,0.151157,7.218342,1117.283333,consumer,2006-01-01
19,19,2006-04,4295868725,0.091841,-0.245961,23.288475,NaN,114.734866,-0.212662,1.235282,1448.236264,consumer,2006-04-01
20,20,2006-07,4295868725,-0.000135,-0.705511,22.818316,NaN,126.740292,0.339104,0.411439,1281.010870,consumer,2006-07-01
21,21,2006-10,4295868725,-0.204360,0.034518,22.852252,NaN,124.101557,-0.047341,0.212121,1145.831522,consumer,2006-10-01
22,22,2007-01,4295868725,-0.203761,-0.006564,22.790050,2324.0,129.003667,-0.421738,0.037840,1103.544444,consumer,2007-01-01


In [9]:
df_esg_3m = pd.read_csv("final data\\df_esg_3m.csv")
df_esg_3m = df_esg_3m[["permid","Yr_Mth","Emissions"]]

df_esg_3m = df_esg_3m.dropna()
df_esg_3m.head()

,permid,Yr_Mth,Emissions
0,4295270870,2000-01,66.400000
1,4295270870,2000-04,62.252747
2,4295270870,2000-07,51.108696
3,4295270870,2000-10,39.630435
4,4295270870,2001-01,35.952941


In [10]:
df_carbon_3m_1 = df_carbon_3m[["Yr_Mth", "mo1_price", "mo3_price", "co1_price", "co12_price", "cbot_eth_q1", "ttf_m1", "ttf_m3"]]
df_gr_f = pd.merge(df_gr, df_carbon_3m_1, left_on="Yr_Mth", right_on = "Yr_Mth")

df_carbon_3m_2 = df_carbon_3m_ret[["Yr_Mth", "mo1_ret", "mo3_ret", "co1_ret", "co12_ret", "cbot_eth_q1_ret", "ttf_m1_ret", "ttf_m3_ret"]]
df_gr_f = pd.merge(df_gr_f, df_carbon_3m_2, left_on="Yr_Mth", right_on = "Yr_Mth")

df_gr_f = df_gr_f.drop("EnvironmentalInnovation", axis=1)
df_gr_f = df_gr_f.rename(columns = {"Emissions":"Emissions_ret" })

df_gr_f = pd.merge(df_gr_f, df_esg_3m, left_on=["permid","Yr_Mth"], right_on = ["permid","Yr_Mth"])
df_gr_f.head()
# Retained Earnings -> req

,Unnamed: 0,Yr_Mth,permid,MomentumL1Y,Mth_Ret,mkt_cap,req,bm,Emissions_ret,buzz,...,ttf_m1,ttf_m3,mo1_ret,mo3_ret,co1_ret,co12_ret,cbot_eth_q1_ret,ttf_m1_ret,ttf_m3_ret,Emissions
0,18,2006-01,4295868725,0.066941,0.101909,22.850595,NaN,123.235305,0.151157,1117.283333,...,NaN,NaN,0.8,0.175924,0.085701,0.092144,NaN,NaN,NaN,63.588889
1,19,2006-04,4295868725,0.091841,-0.245961,23.288475,NaN,114.734866,-0.212662,1448.236264,...,NaN,NaN,0.8,-0.258411,0.121996,0.101705,NaN,NaN,NaN,50.065934
2,20,2006-07,4295868725,-0.000135,-0.705511,22.818316,NaN,126.740292,0.339104,1281.010870,...,NaN,NaN,0.8,-0.162085,0.004840,0.021348,NaN,NaN,NaN,67.043478
3,21,2006-10,4295868725,-0.204360,0.034518,22.852252,NaN,124.101557,-0.047341,1145.831522,...,NaN,NaN,0.8,-0.406425,-0.142722,-0.099146,NaN,NaN,NaN,63.869565
4,22,2007-01,4295868725,-0.203761,-0.006564,22.790050,2324.0,129.003667,-0.421738,1103.544444,...,NaN,NaN,0.8,-0.775334,-0.032819,-0.051143,NaN,NaN,NaN,36.933333


### GMM for Germany 
Fixed effects: industry (green, brown and consumer) + Year (xtset)
(Firm)
Dependent variable: stock returns (monthly for US & quarterly for Germany)
Entity-specific regressor: Emissions, Momentum, Book to market, Mkt cap (heterogenous)
Common regressor: Carbon returns

there is a new Stata command xtreg -vxtdpdgmm
Reference for GMM in https://www.stata.com/manuals13/rgmm.pdf
https://www.statalist.org/forums/forum/general-stata-discussion/general/1567553-s-gmm-interpretation
https://www.statalist.org/forums/forum/general-stata-discussion/general/1395858-xtdpdgmm-new-stata-command-for-efficient-gmm-estimation-of-linear-dynamic-panel-models-with-nonlinear-moment-conditions


            
           

In [11]:
# consider adding investment as one of the moment conditions
# req: retained earnings
list_dict = []
var = ["beta_1","cons_1","beta_2","beta_3","cons_3"]
var_p = ["beta_1_p","cons_1_p","beta_2_p","beta_3_p","cons_3_p"]
for industry in ["brown"]: #,"green","consumer"]:
    dfa1 = df_gr_f[df_gr_f["class"]==industry]  # depends a lot on industry
    stata.pdataframe_to_data(dfa1, force=True)
    try:
        stata.run('''gen Yr_Mth2 = date(Yr_Mth, "YM")''')
        df_dates = pystata.stata.pdataframe_from_data(["Yr_Mth2", "Yr_Mth"])

        #stata.run('''asreg Mth_Ret bm MomentumL1Y mkt_cap Emissions, fmb newey(2) first save(results) se''')
        stata.run('''xtset permid Yr_Mth2''')
        stata.run('''gen lag_mo =mo1_ret[_n-1]''')
        stata.run('''gen lag_emt=Emissions_ret[_n-1]''')
        stata.run('''gen lag_em =(0.01*Emissions[_n-1])''')
        stata.run('''gen lag_cbot=cbot_eth_q1[_n-1]''')
        stata.run('''gen lag_ttf =ttf_m3_ret[_n-1]''')
        stata.run('''gen lag_brent =co12_ret[_n-2]''')  # Brent
        #stata.run('''ds''') #  *lag_em with and without to test robustness ; display variables in memory
        stata.run('''gmm (eq1: Mth_Ret-{beta_1}*lag_em*lag_mo) (eq2: Emissions_ret-{beta_2}*lag_emt-0.005) \
            (eq3: req-{beta_3}*lag_mo), \
            instruments(eq1 eq3: lag_ttf lag_cbot lag_brent) winitial(identity)''')
        #  (eq3: req-{beta_3}*mo1_price - {cons1} ), \
        #stata.run(''' reg req mo1_ret''')  #mo1_ret

        df_vals = pystata.stata.get_return()
        df_vale = pystata.stata.get_ereturn() 
        res  = dict(zip(var, df_vals["r(PT)"][:,0]))
        df_p = dict(zip(var_p, df_vals["r(PT)"][:,3]))
        res.update(df_p)

        res["class"] = industry 
        res["no_of_parameters"] = 5
        res["no_of_moments"] = 9
        res["no_of_obs"] = df_vale["e(N)"]
        
        stata.run('''estat overid''')
        df_valsHS = pystata.stata.get_return()
        res["Hansen_prob"] = df_valsHS["r(J_p)"]
        res["Hansen_J_stat"] = df_valsHS["r(J)"]
        list_dict.append(res)
    except Exception as e: 
        print ("ERROR: " + str(e))
df_res = pd.DataFrame.from_records(list_dict, index = range(len(list_dict)))


Panel variable: permid (unbalanced)
 Time variable: Yr_Mth2, 16802 to 22646, but with gaps
         Delta: 1 unit
(1 missing value generated)
(57 missing values generated)
(1 missing value generated)
(1,186 missing values generated)
(171 missing values generated)
(2 missing values generated)
note: 75 missing values returned for equation 2 at initial values.
note: 268 missing values returned for equation 3 at initial values.

Step 1
Iteration 0:  GMM criterion Q(b) =   11069688  
Iteration 1:  GMM criterion Q(b) =  1604.2981  
Iteration 2:  GMM criterion Q(b) =  1604.2981  

Step 2
Iteration 0:  GMM criterion Q(b) =  .00697961  
Iteration 1:  GMM criterion Q(b) =   .0043495  
Iteration 2:  GMM criterion Q(b) =   .0043495  

GMM estimation 

Number of parameters =   3
Number of moments    =   9
Initial weight matrix: Identity                   Number of obs   =      1,804
GMM weight matrix:     Robust

------------------------------------------------------------------------------
      

In [17]:
df_vals

{'r(PT_has_legend)': 0.0,
 'r(PT_has_cnotes)': 0.0,
 'r(PT_k_ctitles)': 2.0,
 'r(level)': 95.0,
 'r(k_eform)': 1.0,
 'r(PT_rseps)': '`""\' `""\' `""\'',
 'r(PT_rnotes)': '`""\' `""\' `""\'',
 'r(PT_raligns)': '`"right"\' `"right"\' `"right"\'',
 'r(PT_rtitles)': '`"/beta_1"\' `"/beta_2"\' `"/beta_3"\'',
 'r(PT_cformats)': '`"%9.0g"\' `"%9.0g"\' `"%8.2f"\' `"%5.3f"\' `"%9.0g"\' `"%9.0g"\'',
 'r(PT_cspans2)': '`"1"\' `"1"\' `"1"\' `"1"\' `"2"\' `"0"\'',
 'r(PT_ctitles2)': '`"Coefficient"\' `"std. err."\' `"z"\' `"P>|z|"\' `"[95% conf. interval]"\' `""\'',
 'r(PT_cspans1)': '`"1"\' `"1"\' `"1"\' `"1"\' `"1"\' `"1"\'',
 'r(PT_ctitles1)': '`""\' `"Robust"\' `""\' `""\' `""\' `""\'',
 'r(put_tables)': 'PT',
 'r(citype)': 'normal',
 'r(_collect_prefix_get)': 'ignore',
 'r(PT)': array([[1.60762509e-01, 2.14641559e-02, 7.48981278e+00, 6.89719276e-14,
         1.18693537e-01, 2.02831482e-01],
        [9.80381253e-01, 2.64999607e-01, 3.69955738e+00, 2.15975809e-04,
         4.60991567e-01, 1.4997

In [18]:
df_vale

{'e(rank)': 3.0,
 'e(N)': 1804.0,
 'e(Q)': 0.0043495034089505455,
 'e(J)': 7.846504149746784,
 'e(J_df)': 6.0,
 'e(k_1)': 1.0,
 'e(k_2)': 1.0,
 'e(k_3)': 1.0,
 'e(converged)': 1.0,
 'e(has_xtinst)': 0.0,
 'e(type)': 1.0,
 'e(version)': 18.0,
 'e(Q_criterion)': 0.0043495034089505455,
 'e(n_eq)': 3.0,
 'e(k)': 3.0,
 'e(n_moments)': 9.0,
 'e(k_eq)': 3.0,
 'e(k_aux)': 3.0,
 'e(k_eq_model)': 0.0,
 'e(scorevers)': 'version 18:',
 'e(cmdline)': 'gmm (eq1: Mth_Ret-{beta_1}*lag_em*lag_mo) (eq2: Emissions_ret-{beta_2}*lag_emt-0.005) (eq3: req-{beta_3}*lag_mo), instruments(eq1 eq3: lag_ttf lag_cbot lag_brent) winitial(identity)',
 'e(cmd)': 'gmm',
 'e(estat_cmd)': 'gmm_estat',
 'e(predict)': 'gmm_p',
 'e(marginsnotok)': 'Residuals SCores',
 'e(marginsok)': 'xb',
 'e(marginsprop)': 'allcons nochainrule',
 'e(eqnames)': 'eq1 eq2 eq3',
 'e(technique)': 'gn',
 'e(group)': 'permid',
 'e(winit)': 'Identity',
 'e(estimator)': 'twostep',
 'e(wmatrix)': 'robust',
 'e(vce)': 'robust',
 'e(vcetype)': 'Robus

In [19]:
Wt = df_vale['e(S)']
df_wt = pd.DataFrame(Wt)
df_wt.to_clipboard()

In [20]:
conv = df_vale['e(converged)']
conv

1.0

In [22]:
df_res_t = df_res.transpose()
df_res_t = df_res_t.reindex(index_list)
df_res_t.to_clipboard()
df_res_t.head(20)

,0
beta_1,0.160763
beta_1_p,0.0
cons_1,0.980381
cons_1_p,0.000216
beta_2,2212.272684
beta_2_p,0.0
beta_3,NaN
beta_3_p,NaN
cons_3,NaN
cons_3_p,NaN


           #     instruments(1: lag_ttf lag_brent lag_em) \
           # instruments(3: lag_ttf lag_brent lag_em) \
             #   xtinstruments(1: Emissions, lags(2/.)) \
            #xtinstruments(3: Emissions, lags(2/.)) \
            # winitial(xt L L)''')
            #instruments(3 : lag_cbot lag_oil lag_e) winitial(identity)''')
            # instruments(2: lag_ttf ) \
        # a non-concave error is obtained if you do not have as many instruments as parameters fro estimation
        # Need to have more instruments than moments